In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle
import time
import scipy.stats as st
import os 
from lfimodels.balancednetwork.BalancedNetworkSimulator import BalancedNetwork
from lfimodels.balancednetwork.BalancedNetworkStats import BalancedNetworkStats

In [ ]:
inference_method = 'basic'
filename = '15029961421065884_basic_ntrain500'

save_figure = True
path_to_save_folder = 'figures/'

time_str = filename[:filename.find('_')]
filename.find('_')
fullname = 'data/' + filename + '.p'

In [ ]:
dim_params = 4
assert dim_params > 1, 'this notebook is for inference on more than 1 parameter.'
m = BalancedNetwork(dim=dim_params, first_port=8010, verbose=False, n_servers=3)

In [ ]:
# load data 
with open(fullname, 'rb') as handle:
    result_dict = pickle.load(handle)

In [ ]:
# unpack values 
true_params, stats_obs, nrounds, ntrain, posterior = result_dict.values()
mean, std = posterior.values()

## Compare to true parameter 

We have generated the observed data ourselves so we do have the true parameter. The mean of the posterior should be close to it when evaluated for $x=x_{obs}$

In [ ]:
plt.figure(figsize=(15, 10))
theta = np.linspace(0.01, 0.1, 100)
weight_labels = ['$J^{EE}$', '$J^{EI}$', '$J^{IE}$', '$J^{II}$']
plot_idx = 1
for i in range(dim_params): 
    for j in range(dim_params): 
        if i==j: 
            plt.subplot(dim_params, dim_params, plot_idx)
            post_pdf = st.norm.pdf(x=theta, loc=mean[i], scale=std[i, i])
            plt.plot(theta, post_pdf)
            plt.axvline(x=true_params[0][i], color='C1', label=weight_labels[i])
            plt.legend(prop=dict(size=12))
        elif i < j: 
            sub_mean = [mean[i], mean[j]]
            sub_std = sub_std = [[std[i, i], std[i, j]], [std[j, i], std[j, j]]]
            if np.isfinite(sub_std).all(): 
                x, y = np.meshgrid(theta, theta)
                pos = np.dstack((x, y))
                rv = st.multivariate_normal(sub_mean, sub_std)
                plt.subplot(dim_params, dim_params, plot_idx)
                plt.contourf(x, y, rv.pdf(pos))
                plt.plot([true_params[0][i]], [true_params[0][j]], 'o', color='C1')
        plot_idx += 1 

if save_figure and os.path.exists(path_to_save_folder): 
    filename = time_str + '_{}_posteriormatrix_r{}_ntrain{}.pdf'.format(inference_method, nrounds, ntrain)
    plt.tight_layout()
    plt.savefig('figures/' + filename)

## Posterior predictive checking: 

Sample from the estimated posterior and simulate with the sampled parameters.

In [ ]:
post = st.multivariate_normal(mean, std)

In [ ]:
# generate a few samples and simulate 
n_samples = 5 
params = []
for i in range(n_samples): 
    params.append(post.rvs())

In [ ]:
params = [post.mean]
print(params)

In [ ]:
# setup a simulator 
m = BalancedNetwork(n_servers=3, dim=4, duration=3., first_port=8010, 
                   save_raster_plots=True, save_folder='/Users/Jan/Dropbox/Master/mackelab/code/lfi-experiments/balancednetwork/figures/simulation_raster_plots/')

In [ ]:
s = BalancedNetworkStats()

In [ ]:
try: 
    m.start_server()
except: 
    m.stop_server()
    print('server setup failed')

In [ ]:
# simulate 
data = m.gen(params)

In [ ]:
m.stop_server()

In [ ]:
# calculate summary stats
stats = []
for d in data: 
    stats.append(s.calc(d))

In [ ]:
stats = np.array(stats).squeeze()

In [ ]:
stats

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(stats[:, 0:2], 'o-')
plt.plot([0, 0], stats_obs[0][:2], 'o')
plt.subplot(122)
plt.plot(stats[:, 2:], 'o-')
plt.plot([0, 0], stats_obs[0][2:], 'o');

In [ ]:
plt.plot(stats, 'o-')
plt.plot(stats_obs.T, 'o-')